<a href="https://colab.research.google.com/github/liorZucker11/cloud-computing/blob/main/Shablool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports and pips:

In [39]:
!pip install ipywidgets
!pip install IPython
!pip install firebase
!pip install nltk
!pip install matplotlib
!pip install ipympl

In [40]:
%matplotlib widget
from IPython.display import display, Image, HTML
import ipywidgets as widgets
from firebase import firebase
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np
import re
import os
import urllib.request
import requests
import time
import random

#import of ChatBot
import nltk
from nltk.chat.util import Chat, reflections

# **DB** - firebase:

In [41]:
firebase = firebase.FirebaseApplication('https://cloudproject-shablool-default-rtdb.europe-west1.firebasedatabase.app/', None)

"""
  Retrieves 'links' and 'links_counter' data associated with the given key from Firebase index.
  Returns a transposed list or None if the key doesn't exist.
"""
def getKey(key):
  word_data = firebase.get(f'/index/{key}',None)
  if word_data == None:
    return word_data
  tmp = [word_data['links'],word_data['links_counter']]
  transposed = [[row[i] for row in tmp] for i in range(len(tmp[0]))]
  return transposed

#Updates or adds a key-value pair in the Firebase index.
def updateKeyValue(key,value):
  firebase.put("/index/", key, value)

# Deletes a key and all associated values from the Firebase index.
def deleteKey(key):
  firebase.delete('/index', key)

def get_key_with_highest_count():
    # Query to order by 'count' under each word and get the last one (the highest)
    query_result = firebase.get('/index', None, params={'orderBy': '"count"', 'limitToLast': 1})
    if not query_result:
        return "No data found.", 0
    word = list(query_result.keys())[0]
    count = query_result[word]['count']
    return word, count

def get_key_with_lowest_count():
    # Query to order by 'count' under each word and get the first one (the lowest)
    query_result = firebase.get('/index', None, params={'orderBy': '"count"', 'limitToFirst': 1})
    if not query_result:
        return "No data found.", 0
    word = list(query_result.keys())[0]
    count = query_result[word]['count']
    return word, count

def get_all_words_with_counts():
    query_result = firebase.get('/index', None)
    if not query_result:
        return [], []
    word_counts = [(word, details['count']) for word, details in query_result.items()]
    most_frequent = sorted(word_counts, key=lambda x: x[1], reverse=True)[:10]
    least_frequent = sorted(word_counts, key=lambda x: x[1])[:10]
    return most_frequent, least_frequent

def get_random_words():
    query_result = firebase.get('/index', None)
    if not query_result:
        return []
    filtered_words = [(word, details['count']) for word, details in query_result.items()]
    random_words = random.sample(filtered_words, 10)
    return random_words

# Variables and Functions

##Logo and variables:

In [42]:
#Initial Global Variables
container=widgets.Box()
results_output=widgets.Output()

In [43]:
# Corrected URL of the logo stored in GitHub (raw content URL)
logo_url = 'https://raw.githubusercontent.com/liorZucker11/cloud-computing/main/logo.jpg'  # Update this URL

# Download the logo
logo_response = requests.get(logo_url)

logo = widgets.Image()
# Ensure the response was successful
if logo_response.status_code == 200:
    logo = widgets.Image(value=logo_response.content, format='jpg', layout=widgets.Layout(height='100px', width='auto', margin='0 0 20px 0'))
else:
    print("Failed to download the logo.")

# Define the layout for the logo with align_items='center'
logo_layout = widgets.Layout(align_items='center', max_width='200px')
# Apply the logo_layout to the logo widget
logo.layout = logo_layout


## Functions for searching algorithem:

In [44]:
#first level
def sort_links_by_counter(links):
    # Sort the list of lists based on the second element (counter) in each sublist
    sorted_links = sorted(links, key=lambda x: x[1], reverse=True)
    # Extract the first element (link) from each sublist and return as a 1-dimensional list
    return [link[0] for link in sorted_links]

#second level
def filter_common_links(big_data_structure):
    # Step 1: Find common links across all wordDatas
    common_links = set(link for link, _ in big_data_structure[0])  # Initialize with links from the first wordData
    for wordData in big_data_structure[1:]:  # Start from the second wordData
        current_links = set(link for link, _ in wordData)
        common_links.intersection_update(current_links)  # Keep only links that are also in the current wordData
    # Step 2: Filter each wordData to keep only elements with links in common_links
    filtered_data_structure = []
    for wordData in big_data_structure:
        filtered_wordData = [[link, counter] for link, counter in wordData if link in common_links]
        filtered_data_structure.append(filtered_wordData)
    return filtered_data_structure

#third level
def calculate_average_counters(filtered_data_structure):
    link_counters = {}  # Dictionary to hold the total counters for each link and the number of occurrences
    # Aggregate counters for each link
    for wordData in filtered_data_structure:
        for link, counter in wordData:
            if link in link_counters:
                link_counters[link][0] += counter  # Add to the total counter
                link_counters[link][1] += 1  # Increment occurrences
            else:
                link_counters[link] = [counter, 1]  # Initialize with the first counter and set occurrences to 1
    # Calculate average for each link and prepare the result list
    average_counters = [[link, total_counter / occurrences] for link, (total_counter, occurrences) in link_counters.items()]
    # Sort the result list by link for consistent output, if needed
    average_counters.sort(key=lambda x: x[0])
    return average_counters

#fourth level
def sort_by_counters_desc(average_counters):
    # Sort the list by the counter value in descending order
    sorted_average_counters = sorted(average_counters, key=lambda x: x[1], reverse=True)
    return sorted_average_counters

"""
  Searches for links associated with the stemmed words in the query.

  This function takes a search query as input, stems the words using Porter Stemmer,
  retrieves data for each stemmed word from the Firebase index using the getKey function,
  filters common links across all search results, calculates the average counters for each link,
  sorts the links by their counters in descending order, and finally returns a list of links
  extracted from the sorted results.

  If no results are found for the query, it returns a message indicating that the key
  does not exist in the index.
"""
def search(query):
    stemmer = PorterStemmer()
    query = query.lower()
    results = []
    query_words = query.split()
    for word in query_words:
      word = stemmer.stem(word)
      results.append(getKey(word))
    if results == [] or results == [""] or results == [None]:
      return None

    level1 = filter_common_links(results)
    level2 = calculate_average_counters(level1)
    level3 = sort_by_counters_desc(level2)
    return level3

## Functions for admin screen algo:

In [45]:
 # Processes the input value to prepare it for indexing.
def prepare_value_for_index(value):
  #https://portal.azure.com/ 1
  count,links,links_counter = process_data(value)
  val = {"count" : count , "links": links, "links_counter":links_counter}
  return val

    # Processes the input string to extract site URLs and their corresponding numbers.
def process_data(input_string):
  lines = input_string.split('\n')
  sites = []
  numbers = []
  for line in lines:
        site, number = line.strip().split()
        sites.append(site)
        numbers.append(int(number))
  return sum(numbers), sites, numbers

    # Checks if the input value is in the correct format for updating.
def check_Value_to_update(value):
  #https://portal.azure.com/ 1
  pattern = r"^.+ \d+$"
  lines = value.split('\n')
  for line in lines:
      if not re.match(pattern, line.strip()):
          return False
  return True

    # Checks the syntax of the input key.
    # The key syntax is considered invalid if it contains any of the forbidden characters: '.', ',', '$', '#', '[', ']', '/'.
def check_key_syntax(key):
  forbidden_chars = ['.', ',', '$', '#', '[', ']', '/']
  for char in key:
      if char in forbidden_chars:
          return False
  return True
admin_stemmer = PorterStemmer()

#**Screens**

#**Admin Screen:**

In [46]:
def create_admin_screen():
  def on_return_button_clicked(b):
      container.children = []
      container.layout = widgets.Layout()
      Main_Screen()
  # Input text box
  input_text = widgets.Text(
    placeholder='Enter text here...',
    description='Key:',
    disabled=False,
    layout=widgets.Layout(width='100%', margin='0 0 10px 0'),
    style={'description_width': 'initial', 'border': '1px solid black'}
  )

  # Larger text area for displaying results
  results_text = widgets.Textarea(
    placeholder='Results will be shown here...',
    description='Results:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px', margin='0 0 10px 0'),
    style={'description_width': 'initial', 'border': '1px solid black'}
  )

  # Buttons for database operations
  delete_button = widgets.Button(description="Delete", button_style='danger', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
  edit_button = widgets.Button(description="Edit", disabled=True, layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
  add_button = widgets.Button(description="Add", button_style='success', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
  get_button = widgets.Button(description="Get", button_style='info', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
  return_button = widgets.Button(description="Return", button_style='warning', layout=widgets.Layout(border_radius='50%', width='80px', height='50px'))

  # Reset spesific labels to ""
  def reset_labels(num):
    if num == 1:
      comment_string.value = ""
    if num == 2:
      comment_string.value = ""
      results_text.value = ''
    if num == 3:
      results_text.value = ''
    if input_text.value == None or input_text.value == "":
        return 1
    return 0

  # Function to handle return button click
  def on_return_button_clicked(b):
    container.children = []
    container.layout = widgets.Layout()
    Main_Screen()

  # Actions for buttons
  #action for delete button
  def on_delete_button_clicked(b):
    value = input_text.value
    if reset_labels(1) == 1:
      return
    value = admin_stemmer.stem(input_text.value)
    data = getKey(value)
    if data == None:
      comment_string.value = "you cant delete this key becuase it already not there!"
      return
    deleteKey(input_text.value)
    comment_string.value = "delete successful"
    reset_labels(3)

  #action for edit button
  def on_edit_button_clicked(b):
    if reset_labels(1) == 1:
      return
    if results_text.value == None or results_text.value == "":
      comment_string.value =  "you cant edit a key without a value. value for example:https://portal.azure.com/ 1"
      return
    if check_key_syntax(input_text.value) == False:
      comment_string.value =  "['.', ',', '$', '#', '[', ']', '/'] chars cannot be in the key name"
      return
    if(check_Value_to_update(results_text.value) == False):
      comment_string.value =  "value not in the structure like it should. value for example:https://portal.azure.com/ 1"
      return
    val_for_index = prepare_value_for_index(results_text.value)
    updateKeyValue(input_text.value,val_for_index)
    comment_string.value = "update successful"

  #action for add button
  def on_add_button_clicked(b):
    if reset_labels(1) == 1:
      return
    if results_text.value == None or results_text.value == "":
      comment_string.value =  "you cant add a key without a value. value for example:https://portal.azure.com/ 1"
      return
    value = input_text.value
    value = admin_stemmer.stem(input_text.value)
    data = getKey(value)
    if data != None:
      comment_string.value =  "you cant add this key beacuse it already in the index!"
      return
    if check_key_syntax(input_text.value) == False:
      comment_string.value =  "['.', ',', '$', '#', '[', ']', '/'] chars cannot be in the key name"
      return
    if(check_Value_to_update(results_text.value) == False):
      comment_string.value =  "value not in the structure like it should. value for example:https://portal.azure.com/ 1"
      return
    val_for_index = prepare_value_for_index(results_text.value)
    updateKeyValue(input_text.value,val_for_index)
    comment_string.value = "add successful"

  #action for get button
  def on_get_button_clicked(b):
    if reset_labels(2) == 1:
      return
    edit_button.disabled = False
    value = input_text.value
    value = admin_stemmer.stem(input_text.value)
    data = getKey(value)
    if data == None:
      comment_string.value = "The key does not exist in the index."
    else:
      results_text.value = '\n'.join([' '.join(map(str, sub_list)) for sub_list in data])


  # Attach actions to buttons
  delete_button.on_click(on_delete_button_clicked)
  edit_button.on_click(on_edit_button_clicked)
  add_button.on_click(on_add_button_clicked)
  get_button.on_click(on_get_button_clicked)
  return_button.on_click(on_return_button_clicked)


  # Arrange buttons horizontally
  buttons_hbox = widgets.HBox([get_button, add_button, edit_button, delete_button], layout=widgets.Layout(justify_content='flex-start'))

  comment_string = widgets.Label("")

  # Arrange all widgets vertically
  additional_string = widgets.Label("Admin Screen")
  hBox_layout = widgets.Layout(display='flex', justify_content='space-between', align_items='center', width='70%', padding='10px', border='1px solid #ccc')
  hBox = widgets.HBox(children=[return_button,additional_string ,logo], layout=hBox_layout)

  vBox_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='70%', padding='10px', border='1px solid #ccc')
  vBox = widgets.VBox([input_text,comment_string ,results_text, buttons_hbox], layout=vBox_layout)

  container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', justify_content='flex-start', height='100vh')
  container = widgets.Box(children=[hBox, vBox], layout=container_layout)

  # Display everything
  display(container)


#**Statistics Screen:**

In [47]:
def compute_statistics_figures():
    most_frequent, least_frequent = get_all_words_with_counts()
    random_words = get_random_words()
    all_counts = [count for _, count in most_frequent + least_frequent] + [count for _, count in random_words]

    fig, axs = plt.subplots(2, 2, figsize=(10, 10))

    # Most Frequent Words
    words, counts = zip(*most_frequent)
    axs[0, 0].bar(words, counts, color='blue')
    axs[0, 0].set_title('Top 10 Most Frequent Words')
    axs[0, 0].tick_params(axis='x', rotation=45)

    # Least Frequent Words
    words, counts = zip(*least_frequent)
    axs[0, 1].bar(words, counts, color='orange')
    axs[0, 1].set_title('Top 10 Least Frequent Words')
    axs[0, 1].tick_params(axis='x', rotation=45)

    # Random Words Frequencies
    words, counts = zip(*random_words)
    axs[1, 0].bar(words, counts, color='green')
    axs[1, 0].set_title('Frequencies of 10 Random Words')
    axs[1, 0].tick_params(axis='x', rotation=45)

    # Distribution of Word Counts
    axs[1, 1].hist(all_counts, bins=10, range=(0, max(all_counts)), color='purple', alpha=0.7)
    axs[1, 1].set_title('Distribution of Word Counts')

    plt.tight_layout()
    plt.close()

    return fig

statistics_figures = compute_statistics_figures()

In [48]:
def create_statistics_screen():
    def on_return_button_clicked(b):
        container.children = []
        container.layout = widgets.Layout()
        Main_Screen()

    word_with_highest_count, count_high = get_key_with_highest_count()
    most_Freq = widgets.Label(f"Most Frequent Word: '{word_with_highest_count}' with a count of {count_high}")
    word_with_lowest_count, count_low = get_key_with_lowest_count()
    less_Freq = widgets.Label(f"Least Frequent Word:  \n'{word_with_lowest_count}' with a count of {count_low}")

    graph_output = widgets.Output()
    with graph_output:
        display(statistics_figures)

    return_button = widgets.Button(description="Return",
                                   button_style='warning',
                                   layout=widgets.Layout(
                                       border_radius='50%',
                                       width='80px',
                                       height='50px',
                                       margin='5px 5px 15px 5px'
                                   ))
    return_button.on_click(on_return_button_clicked)

    headerLabel = widgets.Label("Statistics Screen")
    # Header row with the logo aligned to the right
    hBox_layout = widgets.Layout(display='flex', justify_content='space-between', align_items='center', width='100%', padding='10px')
    header_row = widgets.HBox([return_button, headerLabel, logo], layout=hBox_layout)

    vbox_layout = widgets.Layout(
        display='flex',
        flex_flow='column',
        width='auto',
        background_color='#f0f0f0',
        padding='20px',
        border='2px solid #ddd',
        border_radius='10px',
        box_shadow='0 2px 4px rgba(0,0,0,0.1)',
        margin='20px'
    )
    vbox = widgets.VBox([header_row, most_Freq, less_Freq, graph_output], layout=vbox_layout)
    container_layout = widgets.Layout(
        display='flex',
        flex_flow='column',
        align_items='center',
        width='100%',
        margin='0 auto'
    )
    container = widgets.Box(children=[vbox], layout=container_layout)

    # Display the elements
    display(container)

# **The ChatBot**

In [49]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Define some patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me ChatBot.', 'I go by the name ChatBot.']),
    (r'(.*) your name?', ['You can call me ChatBot.', 'I go by the name ChatBot.']),
    (r'what is Microsoft Azure?', ['Microsoft Azure is a cloud computing platform by Microsoft.']),
    (r'(.*) cloud computing(.*)', ['Cloud computing is a technology that allows users to access and use computing resources over the internet.']),
    (r'Azure services', ['Microsoft Azure offers a variety of services, including compute, storage, databases, and more.']),
    (r'virtual machines', ['Azure Virtual Machines allow you to run Windows or Linux virtual machines in the cloud.']),
    (r'web development on Azure', ['Azure provides services for web development, including Azure App Service and Azure Functions.']),
    (r'Azure App Service', ['Azure App Service is a fully managed platform for building, deploying, and scaling web apps.']),
    (r'Azure Functions', ['Azure Functions is a serverless compute service that enables you to run event-triggered code without explicitly provisioning infrastructure.']),
    (r'Azure pricing', ['Azure offers flexible pricing options, including pay-as-you-go and reserved instances.']),
    (r'Azure certifications', ['Microsoft offers various Azure certifications, such as Azure Administrator, Azure Developer, and Azure Solutions Architect.']),
    (r'Azure security', ['Azure provides a range of security services and features to help protect your data and applications in the cloud.']),
    (r'Azure networking', ['Azure Networking allows you to securely connect your virtual networks to each other or to your on-premises networks.']),
    (r'Azure storage', ['Azure Storage provides scalable, secure, and durable cloud storage for your data.']),
    (r'Azure AI services', ['Azure offers a range of AI services, including Azure Machine Learning and Azure Cognitive Services, to help you build intelligent applications.']),
    (r'Azure governance', ['Azure Governance helps you manage and enforce policies for your Azure resources to maintain compliance and control costs.']),
    (r'Azure migration', ['Azure provides tools and services to help you migrate your on-premises workloads to the cloud.']),
    (r'Azure monitoring', ['Azure Monitor allows you to monitor the performance and health of your applications and infrastructure in real time.']),
    (r'Azure DevOps', ['Azure DevOps is a set of tools and services for agile development, CI/CD, and collaboration to help you ship software faster and with higher quality.']),
    (r'Azure vs AWS', ['Azure and AWS are both leading cloud providers with their own strengths and features.']),
    (r'Azure vs GCP', ['Azure and Google Cloud Platform (GCP) are both major cloud providers, each with its own set of services and capabilities.']),
    (r'how to deploy a website on Azure', ['You can deploy a website on Azure using services like Azure App Service or Azure Virtual Machines.']),
    (r'how to secure Azure resources', ['Azure provides various security features and best practices to help you secure your resources, such as Azure Security Center and Azure Firewall.']),
    (r'how to scale Azure resources', ['You can scale Azure resources vertically or horizontally based on your applications needs, using tools like Azure Autoscale or Azure Load Balancer.']),
    (r'how to monitor Azure resources', ['Azure offers monitoring solutions like Azure Monitor and Azure Log Analytics to track the performance and health of your resources.']),
    (r'how to optimize costs on Azure', ['You can optimize costs on Azure by using services like Azure Cost Management and Azure Advisor to analyze spending and implement cost-saving measures.']),
    (r'what are Azure regions?', ['Azure regions are geographical areas containing one or more data centers, providing services to nearby areas with low-latency network connections.']),
    (r'what is Azure Marketplace?', ['Azure Marketplace is a digital storefront for finding, trying, and buying software and services certified to run on Azure.']),
    (r'what are Azure Resource Groups?', ['Azure Resource Groups are logical containers that help you manage and organize related Azure resources, such as virtual machines, databases, and storage accounts.']),
    (r'how to automate tasks on Azure?', ['You can automate tasks on Azure using services like Azure Automation, Azure Logic Apps, or Azure Functions, which allow you to create workflows and execute actions based on triggers.']),
    (r'what are Azure Cognitive Services?', ['Azure Cognitive Services are a set of APIs and services that enable developers to build intelligent applications by adding features like vision, speech, language, and decision capabilities.']),
    (r'what is Azure Active Directory?', ['Azure Active Directory is Microsofts cloud-based identity and access management service, providing single sign-on and multi-factor authentication for applications running on Azure and other Microsoft services.']),
    (r'what is Azure SQL Database?', ['Azure SQL Database is a fully managed relational database service provided by Microsoft on Azure, offering built-in high availability, automated backups, and intelligent performance tuning.']),
    (r'what is Azure Kubernetes Service (AKS)?', ['Azure Kubernetes Service is a managed Kubernetes service provided by Microsoft on Azure, allowing you to deploy, manage, and scale containerized applications using Kubernetes.']),
    (r'how to set up a VPN on Azure?', ['You can set up a VPN on Azure using Azure Virtual Network Gateway, which allows you to establish secure connections between your on-premises network and Azure Virtual Network.']),
    (r'how to use Azure Blob Storage?', ['Azure Blob Storage is a scalable object storage service provided by Microsoft on Azure, allowing you to store and manage unstructured data such as documents, images, and videos.']),
    (r'how to deploy a virtual machine on Azure?', ['You can deploy a virtual machine on Azure using the Azure portal, Azure CLI, Azure PowerShell, or Azure Resource Manager templates, specifying configuration details such as size, operating system, and storage.']),
    (r'what is Azure Cosmos DB?', ['Azure Cosmos DB is a globally distributed, multi-model database service provided by Microsoft on Azure, supporting various data models such as document, key-value, graph, and column-family, with automatic scaling and low-latency access.']),
    (r'how to use Azure Functions?', ['Azure Functions is a serverless compute service provided by Microsoft on Azure, allowing you to run code in response to events without managing infrastructure, using triggers and bindings to integrate with other Azure services and external sources.']),
    (r'how to deploy a container on Azure?', ['You can deploy a container on Azure using Azure Container Instances, Azure Kubernetes Service, or Azure App Service, depending on your requirements for scalability, management, and orchestration.']),
    (r'what is Azure Key Vault?', ['Azure Key Vault is a cloud service for securely storing and managing cryptographic keys, secrets, and certificates used by cloud applications and services.']),
    (r'how does Azure handle data encryption?', ['Azure provides encryption at rest and in transit for data stored in Azure services, ensuring that your data remains secure and protected.']),
    (r'what are Azure Logic Apps?', ['Azure Logic Apps is a cloud service that allows you to automate workflows and integrate data, applications, and services across cloud and on-premises environments.']),
    (r'how to use Azure Site Recovery?', ['Azure Site Recovery is a disaster recovery service that allows you to replicate workloads and recover them in the event of an outage, ensuring business continuity.']),
    (r'what are Azure Resource Manager templates?', ['Azure Resource Manager templates are JSON files that define the resources and configuration for your Azure infrastructure, allowing you to deploy and manage resources in a repeatable and consistent manner.']),
    (r'how does Azure support IoT solutions?', ['Azure provides a comprehensive suite of services for building and managing IoT solutions, including IoT Hub, Azure IoT Central, and Azure Sphere.']),
    (r'what is Azure Synapse Analytics?', ['Azure Synapse Analytics is a cloud-based analytics service that allows you to analyze and visualize large volumes of data using both structured and unstructured data sources.']),
    (r'how to use Azure DevTest Labs?', ['Azure DevTest Labs is a service that allows you to quickly create and manage environments for development and testing purposes, providing developers with on-demand access to resources.']),
    (r'what are Azure Blueprints?', ['Azure Blueprints is a service that allows you to define and enforce standards and compliance controls for your Azure environments, enabling you to rapidly deploy and maintain consistent environments.']),
    (r'how does Azure handle compliance certifications?', ['Azure adheres to a wide range of compliance certifications and standards, including ISO, SOC, GDPR, HIPAA, and PCI DSS, ensuring that your data remains compliant and secure.']),
    (r'what are Azure Policy and Azure Role-Based Access Control (RBAC)?', ['Azure Policy is a service that allows you to enforce organizational standards and compliance controls for your Azure resources, while Azure RBAC allows you to manage access to Azure resources based on roles and permissions.']),
    (r'how to use Azure Data Factory?', ['Azure Data Factory is a cloud-based data integration service that allows you to create, schedule, and orchestrate data pipelines for ingesting, preparing, and transforming data from various sources.']),
    (r'what is Azure CDN?', ['Azure Content Delivery Network (CDN) is a global network of servers that delivers web content and applications to users with low latency and high availability, improving the performance and scalability of your applications.']),
    (r'how to use Azure Bastion?', ['Azure Bastion is a managed PaaS service that allows you to securely connect to your Azure virtual machines over SSH or RDP without exposing them to the public internet, providing an additional layer of security for your infrastructure.']),
    (r'what is Azure Data Lake Analytics?', ['Azure Data Lake Analytics is a cloud-based analytics service that allows you to run big data analytics and parallel processing on large volumes of data stored in Azure Data Lake Storage, enabling you to gain valuable insights from your data.']),
    (r'how does Azure support serverless computing?', ['Azure provides a range of serverless computing services, including Azure Functions and Azure Logic Apps, which allow you to run code and automate workflows without managing servers, infrastructure, or scaling concerns.']),
    (r'what is Azure Container Registry?', ['Azure Container Registry is a private registry service that allows you to store and manage Docker container images in Azure, providing a secure and scalable solution for deploying containerized applications.']),
    (r'how to use Azure Load Balancer?', ['Azure Load Balancer is a Layer 4 load balancer service that distributes incoming network traffic across multiple virtual machines or instances to ensure high availability, scalability, and reliability for your applications.']),
    (r'what is Azure Firewall?', ['Azure Firewall is a managed firewall service that allows you to protect your Azure virtual network resources by filtering and controlling inbound and outbound traffic based on rules and policies, providing an additional layer of security for your infrastructure.']),
    (r'how to use Azure Time Series Insights?', ['Azure Time Series Insights is a fully managed analytics service that allows you to explore and analyze time-series data from IoT devices and sensors in real-time, enabling you to gain actionable insights and make informed decisions.']),
    (r'what is Azure API Management?', ['Azure API Management is a fully managed service that allows you to publish, secure, and manage APIs at scale, enabling you to accelerate app development, simplify API management, and drive digital transformation.']),
    (r'how to use Azure Active Directory B2C?', ['Azure Active Directory Business-to-Consumer (Azure AD B2C) is a cloud identity management service that allows you to securely authenticate and authorize external users to access your applications and services, enabling you to provide seamless and personalized experiences for your customers.']),
    (r'what is Azure Cost Management?', ['Azure Cost Management is a service that allows you to monitor, analyze, and optimize your Azure spending and usage, enabling you to control costs, maximize efficiency, and ensure compliance with your budgetary requirements.']),
    (r'how to use Azure Machine Learning Studio?', ['Azure Machine Learning Studio is a collaborative, drag-and-drop tool that allows you to build, train, and deploy machine learning models on Azure, enabling you to leverage the power of machine learning to solve complex business problems and drive innovation.'])
]



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#**ChatBot Screen:**

In [50]:
def create_chatbot_screen():
    chatbot = Chat(patterns)

    return_button = widgets.Button(
    description="Return",
    button_style='warning',
    layout=widgets.Layout(
        border_radius='50%',
        width='80px',
        height='50px',
        margin='0px 5px 15px 5px'
        )
    )


    input_text = widgets.Text(
        value='',
        placeholder='Type your message here',
        disabled=False,
        layout=widgets.Layout(
            width='100%',
            margin='0 0 20px 0'
        )
    )

    output = widgets.Output(layout={'width': '100%', 'height': '300px', 'overflow': 'auto'})

    with output:
        print("ChatBot: Hello! I'm your ChatBot. How can I assist you today?")
        time.sleep(1)

    # Function to handle return button click
    def on_return_button_clicked(b):
        container.children = []
        container.layout = widgets.Layout()
        Main_Screen()

    def on_submit(change):
        user_input = input_text.value
        response = chatbot.respond(user_input)
        if response is None:
            response = "I'm not sure how to respond to that."

        with output:
            print(f"You: {user_input}")
            print(f"ChatBot: {response}")
        time.sleep(1)
        input_text.value = ''

    input_text.on_submit(on_submit)
    return_button.on_click(on_return_button_clicked)

    headerLabel = widgets.Label(value = "ChatBot Screen")
    # Header row with the logo aligned to the right
    hBox_layout = widgets.Layout(display='flex', justify_content='space-between', align_items='center', width='100%', padding='10px')
    header_row = widgets.HBox([return_button, headerLabel, logo], layout=hBox_layout)

    vbox_layout = widgets.Layout(
        display='flex',
        flex_flow='column',
        width='60%',
        margin='0 auto',
        background_color='#f0f0f0',
        padding='20px',
        border='2px solid #ddd',
        border_radius='10px',
        box_shadow='0 2px 4px rgba(0,0,0,0.1)'
    )
    vbox = widgets.VBox([header_row, input_text, output], layout=vbox_layout)
    container_layout = widgets.Layout(
        display='flex',
        flex_flow='column',
        align_items='center',
        width='50%',
        overflow='hidden'
    )
    container = widgets.Box(children=[vbox], layout=container_layout)

    display(container)

#**Search Screen (main):**

In [51]:
def Main_Screen():
    # Button actions
    def statistics_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_statistics_screen()

    def admin_screen_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_admin_screen()

    def chatbot_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_chatbot_screen()

    # Create buttons with actions
    admin_screen_button = widgets.Button(description="Admin Screen", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
    statistics_button = widgets.Button(description="Statistics", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
    chatbot_button = widgets.Button(description="ChatBot", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))

    admin_screen_button.on_click(admin_screen_action)
    statistics_button.on_click(statistics_action)
    chatbot_button.on_click(chatbot_action)

    # Adjust layout for input field, button, and results output
    input_layout = widgets.Layout(width='50%', justify_content='center')
    button_layout = widgets.Layout(width='50%', justify_content='center', margin='10px 0 0 0')
    results_layout = widgets.Layout(border='1px solid black', border_radius='5px', width='90%', margin='20px 0', padding='10px', visibility='hidden')
    # Create the search text box, button, and results output
    search_input = widgets.Text(value='', placeholder='Type here to search', description='', disabled=False, layout=input_layout)
    search_button = widgets.Button(description='SEARCH', button_style='success', tooltip='Click to search', icon='search', layout=button_layout)
    results_output = widgets.Textarea(layout=results_layout, rows=10)

    top_buttons = widgets.HBox([admin_screen_button, statistics_button, chatbot_button], layout=widgets.Layout(justify_content='flex-start', width='100%'))
    vbox_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='70%', background_color='#f0f0f0', padding='20px', border='2px solid #ccc', border_radius='10px')
    vbox = widgets.VBox([top_buttons, logo, search_input, search_button], layout=vbox_layout)
    container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center')
    search_layout = widgets.Layout(display='flex', flex_flow='column', align_items='flex-start', width='70%', background_color='#f0f0f0', padding='20px', border='2px solid #ccc', border_radius='10px')

    # Function to simulate a search operation and display results
    def perform_search(b):
        query = search_input.value.strip()
        if not query:
            return
        results_links = search(query)
        if not results_links:
            search_results_ui = widgets.Label("No results found.")
        else:
            links_widgets = []
            for link_info in results_links:
              link, counter = link_info
              last_header = link.split('/')[-1] if link.split('/')[-1] else link.split('/')[-2]
              links_widgets.append(widgets.HTML(value=f"<a href='{link}' target='_blank' style='text-decoration: none;'><h3>{last_header}</h3></a><span style='font-size: small; color: gray; display: block; margin-top: -15px;'>{link}</span><span style='font-size: small; color: gray; display: block; margin-top: -15px;'>Rank: {counter}</span>"))
            search_results_ui = widgets.VBox(links_widgets, layout=search_layout)

        container.children = [vbox, search_results_ui]

    search_button.on_click(perform_search)

    container = widgets.Box(children=[vbox], layout=container_layout)

    # Display the elements
    display(container)


# **The Website**

In [56]:
Main_Screen()

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

Box(children=(HBox(children=(Button(button_style='warning', description='Return', layout=Layout(height='50px',…

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

Box(children=(HBox(children=(Button(button_style='warning', description='Return', layout=Layout(height='50px',…

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

...

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

Box(children=(VBox(children=(HBox(children=(Button(button_style='warning', description='Return', layout=Layout…

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

Box(children=(VBox(children=(HBox(children=(Button(button_style='warning', description='Return', layout=Layout…

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …